In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [6]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [7]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [8]:
#Convert into categorical variable
y_train_categorical = keras.utils.to_categorical(
    y_train,num_classes = 10, dtype='float32'
)

y_test_categorical = keras.utils.to_categorical(
    y_test,num_classes = 10, dtype ='float32'
)

In [26]:
def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation = "relu"),
        keras.layers.Dense(1000, activation = "relu"),
        keras.layers.Dense(10, activation = "sigmoid")
    ])

    model.compile(optimizer ='SGD',
                 loss = 'categorical_crossentropy',
                 metrics= ['accuracy'])
    return model

In [27]:
#Convert numpy array into tensorflow dataset
#Create an object of tensorflow data type
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

#### Building a Pipeline: Helps to read huge dataset
* It gradually read your data step by step and improves the efficiency of the code
#### Mirror Strategy
* Allows you to split dataset, and send each split to diff GPU

In [28]:
type(train_tf_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [29]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [30]:
strategy.num_replicas_in_sync

1

In [31]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

#prefetch : Optimizes the performance
train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [32]:
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=50)


2023-09-22 19:09:36.260260: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:8"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 10
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
expe

Epoch 1/50


2023-09-22 19:09:36.564887: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


200/200 [==============================] - 9s 44ms/step - loss: 1.9833 - accuracy: 0.2971
Epoch 2/50
200/200 [==============================] - 9s 44ms/step - loss: 1.8164 - accuracy: 0.3637
Epoch 3/50
200/200 [==============================] - 9s 44ms/step - loss: 1.7544 - accuracy: 0.3876
Epoch 4/50
200/200 [==============================] - 9s 45ms/step - loss: 1.7114 - accuracy: 0.4040
Epoch 5/50
  2/200 [..............................] - ETA: 10s - loss: 1.6566 - accuracy: 0.4120

2023-09-22 19:10:13.874863: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


...

200/200 [==============================] - 11s 53ms/step - loss: 1.2282 - accuracy: 0.5746
Epoch 45/50
200/200 [==============================] - 11s 53ms/step - loss: 1.2212 - accuracy: 0.5768
Epoch 46/50
200/200 [==============================] - 10s 52ms/step - loss: 1.2143 - accuracy: 0.5793
Epoch 47/50
  3/200 [..............................] - ETA: 9s - loss: 1.1802 - accuracy: 0.5973 

2023-09-22 19:17:41.681041: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


200/200 [==============================] - 10s 51ms/step - loss: 1.2073 - accuracy: 0.5816
Epoch 48/50
200/200 [==============================] - 10s 52ms/step - loss: 1.2005 - accuracy: 0.5843
Epoch 49/50
200/200 [==============================] - 11s 53ms/step - loss: 1.1939 - accuracy: 0.5868
Epoch 50/50
  3/200 [..............................] - ETA: 9s - loss: 1.1589 - accuracy: 0.6107 

2023-09-22 19:18:13.893918: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


200/200 [==============================] - 11s 53ms/step - loss: 1.1870 - accuracy: 0.5892
